In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
import fasttext
from collections import Counter

[nltk_data] Downloading package punkt to /Library/Frameworks/Python.fr
[nltk_data]     amework/Versions/3.7/lib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Оценка тональности выделенных аспектов

Установка dostoevsky

In [2]:
!pip install dostoevsky

Чтобы всё заработало, необходимо было заранее скачать модель (https://storage.b-labs.pro/models/fasttext-social-network-model.bin) и положить её в папку с тетрадкой.

In [3]:
tokenizer = RegexTokenizer()
fasttext.load_model('./fasttext-social-network-model.bin') 
model_dost = FastTextSocialNetworkModel(tokenizer=tokenizer)

Датафрейм с текстами отзывов и их id

In [4]:
train_texts = pd.read_csv('dev_reviews.txt', delimiter='\t', names=['text_id', 'text'])
train_texts

,text_id,text
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ..."
1,1427,Здравствуйте!Посетили ваше заведение вчера пер...
2,16714,"Были в пятницу (19.03.10), заказывали столик д..."
3,797,"Были в ресторане 2 раза. Один раз днем, все по..."
4,34710,Удивляюсь отзывам про хорошее обслуживание. Бы...
...,...,...
66,9216,Вы брали этот ресторан так как он близко от до...
67,8996,"Были с друзьями в пабе Метрополь, всё очень по..."
68,38299,"Случайно увидели акцию на сайте купонов, решил..."
69,37819,Очень долго выбирали ресторан на Новогодний ка...


Проверка работы dostoevsky (смотрим, как он оценивает тональности целых отзывов

In [5]:
messages = list(train_texts['text'])

results = model_dost.predict(messages, k=3)

for message, sentiment in zip(messages, results):
    print(message, '->', sentiment)

Зашли в"аппетит" случайно.Не смотря на то,что был будний день( вторник 14 сентября) и достаточно много народа, все же решили остаться.нас встретил менеджер- темноволосая стройная девушка, проводила к столу и дала меню...Спустя 5 минут пришла официантка, приняла заказ и удалилась...Посмотрев вокруг мы подумали что ждать придется долго- ресторан был полон, однако мы ошиблись , ожидание было недолгим- вполне приемлимым для любого заведения,за приятной беседой мы не заметили как прошло время...Бизнес ланч, на удивление, оказался очень вкусным и сытным- вполне соответствует своей цене. В общем очень хорошее место, разнообразное вкусное меню, приемлемые цены и высокое качество обслуживания! нам очень все понравилось. спасибо этому заведению! обязательно придем на выходных. -> {'positive': 0.3557848632335663, 'neutral': 0.18714269995689392, 'negative': 0.16027602553367615}
Здравствуйте!Посетили ваше заведение вчера первый раз, проходя мимо никак не получалось зайти, до этого пару раз ужинали 

Датафрейм, в котором уже выделены эксплицитные упоминания аспектов и их категории (их выделяли и записывали в этот файл в другой тетрадке)

In [6]:
train_aspects = pd.read_csv('aspects.txt', delimiter='\t', names=['text_id', 'category', 'mention', 'start', 'end'])
train_aspects

,text_id,category,mention,start,end
0,13823,Food,аппетит,8,15
1,13823,Food,девушка,179,186
2,13823,Food,столу,200,205
3,13823,Whole,меню,213,217
4,13823,Whole,официантка,242,252
...,...,...,...,...,...
1124,11770,Food,блюдо,896,901
1125,11770,Food,рыбой,917,922
1126,11770,Whole,Официанты,938,947
1127,11770,Whole,ресторан,1007,1015


Пытаться определить тональность аспекта просто так без контекста - бесполезно, поскольку в большинстве случаев тональность будет определяться как нейтральная

In [7]:
messages_2 = list(train_aspects['mention'])

results = model_dost.predict(messages_2, k=1)

for message, sentiment in zip(messages_2, results):
    print(message, '->', sentiment)

аппетит -> {'skip': 0.1097157895565033}
девушка -> {'neutral': 0.9748311638832092}
столу -> {'neutral': 0.9989450573921204}
меню -> {'neutral': 0.9566442370414734}
официантка -> {'neutral': 1.0000100135803223}
заказ -> {'neutral': 1.0000100135803223}
ресторан -> {'neutral': 1.0000100135803223}
заведения -> {'neutral': 0.348655104637146}
цене -> {'neutral': 0.9995517134666443}
меню -> {'neutral': 0.9566442370414734}
цены -> {'neutral': 1.0000100135803223}
обслуживания -> {'neutral': 0.9566442370414734}
заведению -> {'neutral': 0.348655104637146}
заведение -> {'neutral': 0.7879412174224854}
обслуживанием -> {'neutral': 0.6723417043685913}
посещение -> {'neutral': 1.0000100135803223}
заведения -> {'neutral': 0.348655104637146}
вечером -> {'neutral': 1.0000100135803223}
веранде -> {'neutral': 1.0000100135803223}
ребенком -> {'skip': 0.07586817443370819}
столов -> {'neutral': 1.0000100135803223}
столика -> {'neutral': 1.0000100135803223}
девушка -> {'neutral': 0.9748311638832092}
меню -> {'

лосось -> {'neutral': 0.9956784844398499}
кусок -> {'neutral': 0.9995794296264648}
еды -> {'neutral': 0.9955416321754456}
этаже -> {'neutral': 0.994098961353302}
кальян -> {'neutral': 0.998599648475647}
праздник -> {'speech': 0.754925012588501}
обстановке -> {'neutral': 0.9706977605819702}
двери -> {'neutral': 1.0000100135803223}
Интерьер -> {'neutral': 0.993317186832428}
ресторана -> {'neutral': 0.9988405108451843}
дизайн -> {'neutral': 0.9949496984481812}
вкусом -> {'neutral': 0.9416641592979431}
мелочей -> {'neutral': 0.9926641583442688}
девушка -> {'neutral': 0.9748311638832092}
пасты -> {'neutral': 0.9886782765388489}
приготовления -> {'neutral': 0.4532718360424042}
блюда -> {'neutral': 0.997758150100708}
Меню -> {'neutral': 0.9566442370414734}
ресторана -> {'neutral': 0.9988405108451843}
пасты -> {'neutral': 0.9886782765388489}
Дизайн -> {'neutral': 0.9949496984481812}
меню -> {'neutral': 0.9566442370414734}
крем -> {'neutral': 0.9995371699333191}
суп -> {'neutral': 0.99581122398

цены -> {'neutral': 1.0000100135803223}
утку -> {'neutral': 1.0000100135803223}
порции -> {'neutral': 1.0000100135803223}
суп -> {'neutral': 0.9958112239837646}
Пиво -> {'neutral': 0.5156299471855164}
обслуживание -> {'neutral': 0.91965252161026}
девушки -> {'neutral': 0.9124462008476257}
соус -> {'neutral': 1.0000100135803223}
Интерьер -> {'neutral': 0.993317186832428}
зала -> {'neutral': 0.8354935646057129}
шашлыки -> {'neutral': 0.9399233460426331}
овощи -> {'neutral': 0.9947898983955383}
картофель -> {'neutral': 1.0000100135803223}
морсу -> {'neutral': 0.1968362182378769}
вина -> {'neutral': 1.0000100135803223}
шашлык -> {'neutral': 0.9924327731132507}
Овощи -> {'neutral': 0.9947898983955383}
кухня -> {'neutral': 0.9994029402732849}
Интерьер -> {'neutral': 0.993317186832428}
счет -> {'neutral': 1.0000100135803223}
стены -> {'neutral': 1.0000100135803223}
ощущение -> {'neutral': 0.998309314250946}
Обслуживание -> {'neutral': 0.91965252161026}
заказе -> {'neutral': 1.0000100135803223

Попробуем как-то учесть контекст. Будем основываться не на всём отзыве, а на какой-то его части - на предложении, в котором встретилось конкретное эксплицитное упоминание аспекта. 

У нас в таблице есть индексы начала и конца упоминания аспекта. Будем раздвигать индексы до тех пор, пока не дойдем до границ предложения. А дальше будем оценивать тональность полученного предложения (предложения, оказавшегося между индексами).

In [8]:
punct = ['.', '!', '?']

In [9]:
sent_2 = []
for row in train_aspects.itertuples():
    left_index = row.start
    right_index = row.end
    text_index = row.text_id
    for row2 in train_texts.itertuples():
        if text_index == row2.text_id:
            text = row2.text
    while (text[left_index] not in punct or text[right_index] not in punct) and right_index != len(text)-1:
        if text[left_index] not in punct:
            left_index -= 1
        if text[right_index] not in punct:
            right_index += 1
    sent = text[left_index+1:right_index+1]
    result = model_dost.predict([sent], k=1)
    sent_2.append(list(result[0].keys())[0])

Заменим все 'skip' и 'speech' на нейтральную тональность

In [10]:
for index in range(len(sent_2)):
    if sent_2[index] == 'skip' or sent_2[index] == 'speech':
        sent_2[index] = 'neutral'

Добавим в таблицу

In [11]:
train_aspects['sentiment'] = sent_2

In [12]:
train_aspects

,text_id,category,mention,start,end,sentiment
0,13823,Food,аппетит,8,15,negative
1,13823,Food,девушка,179,186,neutral
2,13823,Food,столу,200,205,neutral
3,13823,Whole,меню,213,217,neutral
4,13823,Whole,официантка,242,252,neutral
...,...,...,...,...,...,...
1124,11770,Food,блюдо,896,901,neutral
1125,11770,Food,рыбой,917,922,neutral
1126,11770,Whole,Официанты,938,947,neutral
1127,11770,Whole,ресторан,1007,1015,neutral


In [13]:
set(list(train_aspects['sentiment']))

{'negative', 'neutral', 'positive'}

Запишем получившуюся таблицу в файл

In [14]:
train_aspects.to_csv(r'./aspect_sentiment.txt', header=None, index=None, sep='\t', mode='a')

### Теперь сделаем оценку тональности по категориям

Создадим вспомогательный словарь (вложенный).

Вид словаря: 

{id текста : {категория: список всех встретившихся тональностей упоминаний аспектов данной категории}}

In [15]:
text_ids_unique = list(set(train_aspects['text_id']))

In [16]:
dict_help = dict.fromkeys(text_ids_unique)

In [17]:
for ids in text_ids_unique:
    new_dict = {'Interior': [],
                'Food': [],
                'Service': [], 
                'Price': [],
                'Whole': []}
    for row in train_aspects.itertuples():
        if ids == row.text_id:
            new_dict[row.category].append(row.sentiment)
    dict_help[ids] = new_dict

In [18]:
dict_help

{9216: {'Interior': ['positive'],
  'Food': ['negative',
   'negative',
   'neutral',
   'neutral',
   'neutral',
   'negative'],
  'Service': ['positive'],
  'Price': [],
  'Whole': ['neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral']},
 13186: {'Interior': ['positive', 'positive'],
  'Food': ['neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral',
   'neutral'],
  'Service': [],
  'Price': [],
  'Whole': ['positive', 'neutral', 'neutral', 'neutral', 'neutral']},
 33798: {'Interior': ['positive'],
  'Food': ['neutral', 'neutral', 'negative', 'positive'],
  'Service': [],
  'Price': [],
  'Whole': ['neutral', 'neutral', 'neutral', 'neutral', 'neutral']},
 3976: {'Interior': ['positive'],
  'Food': ['neutral', 'neutral', 'neutral', 'positive'],
  'Service': ['neutral', 'neutral', 'neutral'],
  'Price': [],
  'Whole': ['neut

Поменяем формат словаря так, чтобы из него можно было сделать датафрейм: ключи - названия колонок, значения - списки значений колонок

In [19]:
dict_df ={'text_id': [],
          'category': [],
          'sentiment': []}
for ids in dict_help.keys():
    for cat in dict_help[ids]:
        dict_df['text_id'].append(ids)
        dict_df['category'].append(cat)
        if dict_help[ids][cat] == []:
            dict_df['sentiment'].append('absence')
        else:
            dict_df['sentiment'].append(Counter(dict_help[ids][cat]).most_common(1)[0][0])

In [20]:
dict_df

{'text_id': [9216,
  9216,
  9216,
  9216,
  9216,
  13186,
  13186,
  13186,
  13186,
  13186,
  33798,
  33798,
  33798,
  33798,
  33798,
  3976,
  3976,
  3976,
  3976,
  3976,
  34956,
  34956,
  34956,
  34956,
  34956,
  11532,
  11532,
  11532,
  11532,
  11532,
  37391,
  37391,
  37391,
  37391,
  37391,
  7824,
  7824,
  7824,
  7824,
  7824,
  785,
  785,
  785,
  785,
  785,
  1427,
  1427,
  1427,
  1427,
  1427,
  33043,
  33043,
  33043,
  33043,
  33043,
  34710,
  34710,
  34710,
  34710,
  34710,
  30744,
  30744,
  30744,
  30744,
  30744,
  2073,
  2073,
  2073,
  2073,
  2073,
  1434,
  1434,
  1434,
  1434,
  1434,
  7193,
  7193,
  7193,
  7193,
  7193,
  31004,
  31004,
  31004,
  31004,
  31004,
  797,
  797,
  797,
  797,
  797,
  11421,
  11421,
  11421,
  11421,
  11421,
  36126,
  36126,
  36126,
  36126,
  36126,
  27040,
  27040,
  27040,
  27040,
  27040,
  3231,
  3231,
  3231,
  3231,
  3231,
  36512,
  36512,
  36512,
  36512,
  36512,
  5155,
  5155

Создаем датафрейм

In [21]:
categories_sentiments = pd.DataFrame.from_dict(dict_df)

In [22]:
categories_sentiments

,text_id,category,sentiment
0,9216,Interior,positive
1,9216,Food,negative
2,9216,Service,positive
3,9216,Price,absence
4,9216,Whole,neutral
...,...,...,...
350,13823,Interior,absence
351,13823,Food,neutral
352,13823,Service,neutral
353,13823,Price,neutral


Записываем в файл

In [23]:
categories_sentiments.to_csv(r'./categories_sentiments.txt', header=None, index=None, sep='\t', mode='a')